In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level=logging.INFO)

from abc import ABC, abstractmethod
from contextlib import AbstractContextManager
from typing import Protocol, runtime_checkable

import torch
from torch import Tensor, jit, nn, tensor
from torch.linalg import matrix_norm

# from linodenet.lib import singular_triplet
from torchinfo import summary

# from linodenet.parametrize import Parametrization, SpectralNormalization

In [ ]:
import inspect

In [ ]:
import torch

In [ ]:
t = torch.tensor(1.23)

In [ ]:
inspect.signature(t.uniform_)

In [ ]:
inspect.signature(torch.nn.Parameter.uniform_)

In [ ]:
def uniform_(self, **_):
    pass

In [ ]:
uniform_(from=0)

In [ ]:
A = nn.Parameter(
    tensor(
        [
            [-0.0037, 0.2682, -0.4115, -0.3680],
            [-0.1926, 0.1341, -0.0099, 0.3964],
            [-0.0444, 0.1323, -0.1511, -0.0983],
            [-0.4777, -0.3311, -0.2061, 0.0185],
        ]
    )
)

A_t = A.T
u = torch.tensor([-0.5966, 0.1820, -0.8567, 1.1006])
v = torch.tensor([-0.5966, 0.1820, -0.8567, 1.1006])

In [ ]:
nn.Parameter(A)

In [ ]:
u = A.mv(v)

In [ ]:
u /= u.norm()

In [ ]:
v = A_t.mv(u)
v /= v.norm();

In [ ]:
u, v

In [ ]:
(A @ v).norm()

In [ ]:
model = nn.Linear(4, 4)

self = SpectralNormalization(model.weight)

# model.weight = spec.weight
# model.spec = spec
new_tensors = self.forward()

In [ ]:
new_tensors = self.forward()

In [ ]:
spec.register_buffer("test", None, persistent=True)

In [ ]:
spec.weight, spec.u, spec.v, spec.sigma

In [ ]:
spec.u = None

In [ ]:
self = spec
new_tensors = self.forward()

# copy the new tensors into the cache
for key, tensor in new_tensors.items():
    self.cached_tensors[key].copy_(tensor)

In [ ]:
new_tensors = spec.forward()

In [ ]:
spec.cached_tensors["u"].copy_(new_tensors["u"])

In [ ]:
spec.recompute_cache()

In [ ]:
jit.save(jit.script(spec), "foo")

In [ ]:
spec = jit.load("foo")

In [ ]:
spec.cached_tensors

In [ ]:
model

In [ ]:
matrix_norm(spec.cached_weight, ord=2)

In [ ]:
?norm

In [ ]:
model.weight.norm(p=2)

In [ ]:
spec.cached_tensors

In [ ]:
spec.parametrized_tensors

In [ ]:
raise

In [ ]:
spec.recompute_cache()

In [ ]:
raise

In [ ]:
model.weight.norm(p=2)